In [1]:
import numpy as np
import pandas as pd

In [24]:
aDF = pd.read_csv("a_win7_proc2.csv")

In [25]:
uniq_ops = {}
for col_name in list(aDF.columns):
    if col_name in ["Time of Day","Process Name","PID"]:  #skip these
        continue
    _tmp = aDF.loc[:,col_name].unique()
    print(f"{col_name}: {_tmp}")
    uniq_ops[col_name] = _tmp
print(uniq_ops.keys())

Operation: ['Process Start' 'Thread Create' 'Load Image' 'CreateFile' 'RegOpenKey'
 'RegQueryValue' 'RegCloseKey' 'QueryBasicInformationFile' 'CloseFile'
 'CreateFileMapping' 'QueryStandardInformationFile' 'RegCreateKey'
 'RegSetValue' 'RegEnumValue' 'RegQueryKey' 'RegEnumKey'
 'FileSystemControl' 'QueryDirectory' 'ReadFile' 'QuerySecurityFile'
 'QueryNameInformationFile' 'Process Create' 'QueryAttributeTagFile'
 'SetDispositionInformationFile' 'Thread Exit' 'Process Exit']
Path: [nan
 'C:\\Users\\IEUser\\Desktop\\Assignment 1(2)\\Assignment 1\\Malware samples\\Sample_A\\Sample A -\\server.exe'
 'C:\\Windows\\System32\\ntdll.dll'
 'C:\\Windows\\Prefetch\\SERVER.EXE-56F6A005.pf'
 'HKLM\\System\\CurrentControlSet\\Control\\Session Manager'
 'HKLM\\System\\CurrentControlSet\\Control\\Session Manager\\CWDIllegalInDLLSearch'
 'C:\\Users\\IEUser\\Desktop\\Assignment 1(2)\\Assignment 1\\Malware samples\\Sample_A\\Sample A -'
 'C:\\Windows\\System32\\kernel32.dll'
 'C:\\Windows\\System32\\Kern

### Registry Events

In [26]:
def see_events(df,ops_list,pid=None):
    for op in ops_list:
        print("\x1b[31m==========================\x1b[0m",op,"\x1b[31m===========================\x1b[0m")#"\x1b[31m\"red\"\x1b[0m"
        Events = df[(df["Operation"] == op)]
        if pid == None:
            for i in range(len(Events)):
                print(f"{Events.iloc[i,1]} --{Events.iloc[i,3]} -- {Events.iloc[i,4]} -- {Events.iloc[i,6]}")
        else:
            for i in range(len(Events)):
                if Events.iloc[i,1] == pid:
                    print(f"{Events.iloc[i,1]} --{Events.iloc[i,3]} -- {Events.iloc[i,4]} -- {Events.iloc[i,6]}")
#To see all registry events, uncomment the below
#see_events(aDF,[ops for ops in uniq_ops["Operation"] if ops.startswith("Reg")])
#The only interesting ones are RegSetValue
see_events(aDF,["RegSetValue"],"server.exe")

========================== RegSetValue ===========================
server.exe --RegSetValue -- HKCU\Software\XtremeRAT\Mutex -- Type: REG_EXPAND_SZ, Length: 34, Data: BlackMamba_Mutex


### File Events

In [27]:
#To see all file events, uncomment the below
#see_events(aDF,[ops for ops in uniq_ops["Operation"] if ops.endswith("File")])
#The only interesting ones are RegSetValue
see_events(aDF,["CreateFile"])

========================== CreateFile ===========================
server.exe --CreateFile -- C:\Windows\Prefetch\SERVER.EXE-56F6A005.pf -- Desired Access: Generic Read, Disposition: Open, Options: Synchronous IO Non-Alert, Attributes: n/a, ShareMode: None, AllocationSize: n/a
server.exe --CreateFile -- C:\Users\IEUser\Desktop\Assignment 1(2)\Assignment 1\Malware samples\Sample_A\Sample A - -- Desired Access: Execute/Traverse, Synchronize, Disposition: Open, Options: Directory, Synchronous IO Non-Alert, Attributes: n/a, ShareMode: Read, Write, AllocationSize: n/a, OpenResult: Opened
server.exe --CreateFile -- C:\Windows\System32\sechost.dll -- Desired Access: Read Attributes, Disposition: Open, Options: Open Reparse Point, Attributes: n/a, ShareMode: Read, Write, Delete, AllocationSize: n/a, OpenResult: Opened
server.exe --CreateFile -- C:\Windows\System32\sechost.dll -- Desired Access: Read Data/List Directory, Execute/Traverse, Synchronize, Disposition: Open, Options: Synchronous IO N

In [28]:
FileCreateEvents = aDF[(aDF["Operation"] == "CreateFile")]
for i in range(len(FileCreateEvents)):
    file_created = FileCreateEvents.iloc[i,4]
    for file_ext in ["exe","cfg","dat","html"]:
        if file_created.endswith(file_ext):
            print(f"\x1b[31m{file_ext}\x1b[0m ",end='')
            print(f"{FileCreateEvents.iloc[i,3]} -- {FileCreateEvents.iloc[i,4]} -- {FileCreateEvents.iloc[i,6]}")
    print({FileCreateEvents.iloc[i,4]} )

{'C:\\Windows\\Prefetch\\SERVER.EXE-56F6A005.pf'}
{'C:\\Users\\IEUser\\Desktop\\Assignment 1(2)\\Assignment 1\\Malware samples\\Sample_A\\Sample A -'}
{'C:\\Windows\\System32\\sechost.dll'}
{'C:\\Windows\\System32\\sechost.dll'}
{'C:\\Users\\IEUser\\Desktop\\Assignment 1(2)\\Assignment 1\\Malware samples\\Sample_A\\Sample A -\\version.DLL'}
{'C:\\Windows\\System32\\version.dll'}
{'C:\\Windows\\System32\\version.dll'}
{'C:\\Windows\\System32\\imm32.dll'}
{'C:\\Windows\\System32\\imm32.dll'}
{'C:\\Windows\\System32\\imm32.dll'}
{'C:\\Windows\\System32\\imm32.dll'}
{'C:\\Windows\\System32\\imm32.dll'}
{'C:\\Windows\\System32\\imm32.dll'}
{'C:\\Windows\\Globalization\\Sorting\\SortDefault.nls'}
{'C:\\Users\\IEUser\\AppData\\Roaming'}
{'C:\\Windows\\System32\\rpcss.dll'}
{'C:\\Windows\\System32\\rpcss.dll'}
{'C:\\Windows\\System32\\rpcss.dll'}
{'C:\\Windows\\System32\\rpcss.dll'}
{'C:\\Users\\IEUser\\Desktop\\Assignment 1(2)\\Assignment 1\\Malware samples\\Sample_A\\Sample A -\\CRYPTBASE.dl

In [12]:
delete_events = aDF[aDF["Operation"]=='SetEndOfFileInformationFile']
delete_events

,Time of Day,Process Name,PID,Operation,Path,Result,Detail
